In [ ]:
import torch
import torch.nn.functional as F
import os
from collections import defaultdict
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Number of candidate models
num_candidates = 30
top_k = 10  # Select Top-K models


# Format: [(model_id, accuracy_score)]
model_scores = [
    # Populate with actual accuracy scores, e.g.,
    (1, 85.2),
    (2, 88.5),
    # ... up to model_id=30
]


def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion * planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion * planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x, return_features=False):
        x = self.conv1(x)
        x = self.bn1(x)
        out = F.relu(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.adaptive_avg_pool2d(out, (1, 1))
        feature = out.view(out.size(0), -1)
        out = self.linear(feature)

        if return_features:
            return out, feature
        else:
            return out


def resnet18(num_classes=100):
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes)


def resnet34(num_classes=10):
    return ResNet(BasicBlock, [3, 4, 6, 3], num_classes)


def resnet50(num_classes=10):
    return ResNet(Bottleneck, [3, 4, 6, 3], num_classes)


def resnet101(num_classes=10):
    return ResNet(Bottleneck, [3, 4, 23, 3], num_classes)


def resnet152(num_classes=10):
    return ResNet(Bottleneck, [3, 8, 36, 3], num_classes)

class CNNCifar(nn.Module):
    def __init__(self):
        super(CNNCifar, self).__init__()
        self.conv1 = nn.Conv2d(3, 128, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(128, 128, 3)
        self.conv3 = nn.Conv2d(128, 128, 3)
        self.fc1 = nn.Linear(128 * 4 * 4, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = x.view(-1, 128 * 4 * 4)
        x = self.fc1(x)
        return x



# Example: Assuming you have validation data loaded as tensors
# Replace 'path_to_validation_data.pt' and 'path_to_validation_targets.pt' with actual paths
validation_data = torch.load('path_to_validation_data.pt')  # Replace with actual path
validation_targets = torch.load('path_to_validation_targets.pt')  # Replace with actual path

validation_dataset = TensorDataset(validation_data, validation_targets)
validation_dataloader = DataLoader(validation_dataset, batch_size=128, shuffle=False)


def extract_model_parameters(model_path):
    """Load model and extract flattened, normalized parameter vector."""
    if not os.path.exists(model_path):
        print(f"Warning: Model file {model_path} not found! Skipping.")
        return None

    model = resnet18(num_classes=100).to(device)
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint)
    model.eval()

    # Flatten all parameters into a single vector
    params = [param.view(-1) for param in model.parameters()]
    param_vector = torch.cat(params)

    # Normalize the parameter vector
    param_vector = F.normalize(param_vector, p=2, dim=0)

    return model, param_vector


# Define the path to the local model (use a distinct filename to avoid duplication)
local_model_path = f'./pruned_models/pruned_client_local.pth'
local_model_id = -1  # Unique identifier for the local model
local_model, local_model_params = extract_model_parameters(local_model_path)
if local_model_params is None:
    raise ValueError("Failed to extract local model parameters!")

# **Normalize local model parameters for consistency**
local_model_params = F.normalize(local_model_params, p=2, dim=0)

# Compute actual validation accuracy for the local model
def compute_validation_accuracy(model, dataloader, device):
    """Compute validation accuracy of the model on the given dataloader."""
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
    accuracy = 100 * correct / total
    return accuracy

local_model_score = compute_validation_accuracy(local_model, validation_dataloader, device)

# Add local model to model_scores
model_scores.append((local_model_id, local_model_score))


model_parameters = {}
candidate_models = {}

for model_id in range(1, num_candidates + 1):
    # No need to check for model_id ==0 as local_model_id is -1
    model_path = f'./pruned_models/pruned_client_{model_id}.pth'
    model, params = extract_model_parameters(model_path)
    if params is not None:
        model_parameters[model_id] = params
        candidate_models[model_id] = model
    else:
        print(f"Warning: Parameters for model_id={model_id} could not be loaded.")

# Add local model parameters explicitly
model_parameters[local_model_id] = local_model_params
candidate_models[local_model_id] = local_model  # Ensures the local model is included if needed


if not model_scores:
    raise ValueError("model_scores is empty! Ensure validation accuracy scores are provided.")

# Sort model_scores in descending order based on accuracy
model_scores_sorted = sorted(model_scores, key=lambda x: x[1], reverse=True)


ranked_models = defaultdict(list)
for model_id, score in model_scores_sorted:
    ranked_models[score].append(model_id)


top_k_models = []
for score in sorted(ranked_models.keys(), reverse=True):
    tied_models = ranked_models[score]

    if len(tied_models) == 1:
        top_k_models.append(tied_models[0])
    else:
        # Compute cosine similarity with the local model
        similarities = []
        for model_id in tied_models:
            model_params = model_parameters.get(model_id)
            if model_params is None:
                continue  # Skip if parameters are not loaded
            cos_sim = F.cosine_similarity(
                local_model_params.unsqueeze(0), 
                model_params.unsqueeze(0), 
                dim=1
            ).item()
            similarities.append((model_id, cos_sim))

        # Sort by cosine similarity in descending order
        similarities_sorted = sorted(similarities, key=lambda x: x[1], reverse=True)

        # Add sorted models to Top-K selection
        for model_id, _ in similarities_sorted:
            top_k_models.append(model_id)

    # Stop when we have Top-K models
    if len(top_k_models) >= top_k:
        break


top_k_models = top_k_models[:top_k]

# Ensure we have enough models
if len(top_k_models) < top_k:
    raise ValueError(f"Only {len(top_k_models)} models available, but Top-K requires {top_k}!")

print(f"Selected Top-{top_k} Models:", top_k_models)
